File Preprocessing

In [77]:
!pip install -U numpy
!pip install pyldavis

Requirement already up-to-date: numpy in /opt/conda/lib/python3.7/site-packages (1.16.3)


In [1]:
import os
import codecs
import spacy
import numpy
import gensim
from spacy import displacy
import pandas as pd
import itertools as it

In [2]:
# PICK THE LANGUAGE OF THE CORPUS YOU WISH TO EXAMINE
language = 'de'

In [3]:
%%time
if language == 'fr':
    print('loading french model')
    nlp = spacy.load("fr_core_news_md")
elif language == 'de':
    print ('loading german model')
    nlp = spacy.load("fr_core_news_md")   


loading german model
CPU times: user 23.7 s, sys: 550 ms, total: 24.3 s
Wall time: 24.2 s


We open the preprocessed de file

In [7]:
directory = "../corpus_samples/processed/"
filename = directory + language +'.txt'

with codecs.open(filename, encoding='utf_8') as f:
    text = f.readlines()
print(text)

["auf Sternwarte Wien ander durch VON Oarr auf Sternwarte München ausgeführt worden war Das durch Konstanten gramme val abs Wien M.-G. i. m cm sek-2 definiert Wiener gramme System ist bisher allgemein al Grundlage für relativen gramme Messungen beibehalten worden näherungsweise ermittelten Reduktionsbeträge Beobachtungsreihen auf dasselbe haben Herrn Prof helmert dazu gedient eine diesem System angepaßte Normalformel abzuleiten seit zur Darstellung Verlaufs Schwerkraft an Erdoberfläche verwendet wird In oben erwähnten Berichte wie Herr Prof helmert jedoch schon darauf hin daß Wiener Fundamentalkonstante nach seinen auf andere Material gestützten Untersuchungen wahrscheinlich einer Korrektion cm sek-2 bedürfe ihm älteren meist beträchtlichen systematischen Fehlern behafteten absoluten Messungen zur Ableitung eines Normalwertes gramme nicht geeignet erschienen so leitete er im jahre eine absolute Bestimmung gramme für Potsdam Wege bei dieser über jahre ausgedehnten besonder auf eliminati

In [438]:
%%time
doc = nlp(text)

CPU times: user 200 ms, sys: 20 ms, total: 220 ms
Wall time: 215 ms


In [441]:
my_stop_words = {}
my_stop_words['de'] = [u'herr', u'mit',u'der',u'die',u'das', u'in',u'sein',u'dies',u'einen',u'und', u'ich', u'mit',u'zu', u'von',u'bis', u'S.', u'°', u'den', u'dem']
my_stop_words['fr'] = [u'du', u'de',u'un']

for stopword in my_stop_words[language]:
    lexeme = nlp.vocab[stopword]
    print(lexeme.text)
    lexeme.is_stop = True


herr
mit
der
die
das
in
sein
dies
einen
und
ich
mit
zu
von
bis
S.
°
den
dem


In [442]:
tokens = [token.lemma_ for token in doc if not token.is_space and not token.is_stop and not token.is_punct and not token.like_num and not token.is_space]
tokens

[auf,
 Sternwarte,
 Wien,
 andere,
 durch,
 VON,
 Oarr,
 auf,
 Sternwarte,
 München,
 ausgeführt,
 worden,
 war,
 Das,
 durch,
 Konstanten,
 g,
 val,
 abs,
 Wien,
 M.-G.,
 I.,
 m,
 cm,
 sek-2,
 definierte,
 Wiener,
 g,
 System,
 ist,
 bisher,
 allgemein,
 als,
 Grundlage,
 für,
 relativen,
 g,
 Messungen,
 beibehalten,
 worden,
 näherungsweise,
 ermittelten,
 Reduktionsbeträge,
 Beobachtungsreihen,
 auf,
 dasselbe,
 haben,
 Herrn,
 Prof,
 HELMERT,
 dazu,
 gedient,
 eine,
 diesem,
 System,
 angepaßte,
 Normalformel,
 abzuleiten,
 seit,
 zur,
 Darstellung,
 Verlaufs,
 Schwerkraft,
 an,
 Erdoberfläche,
 verwendet,
 wird,
 In,
 oben,
 erwähnten,
 Berichte,
 wies,
 Herr,
 Prof,
 HELMERT,
 jedoch,
 schon,
 darauf,
 hin,
 daß,
 Wiener,
 Fundamentalkonstante,
 nach,
 seinen,
 auf,
 anderes,
 Material,
 gestützten,
 Untersuchungen,
 wahrscheinlich,
 einer,
 Korrektion,
 cm,
 sek-2,
 bedürfe,
 ihm,
 älteren,
 meist,
 beträchtlichen,
 systematischen,
 Fehlern,
 behafteten,
 absoluten,
 Messungen,

In [443]:
token_text = [token.orth_ for token in tokens]
token_text
token_pos = [token.pos_ for token in tokens]
token_pos
token_ent_type = [token.ent_type_ for token in tokens]
token_ent_type
pd.DataFrame(zip(token_text, token_ent_type), columns= ['text', 'ent type'])

,text,ent type
0,auf,
1,Sternwarte,
2,Wien,
3,andere,
4,durch,
5,VON,PER
6,Oarr,PER
7,auf,PER
8,Sternwarte,PER
9,München,LOC


In [444]:
from gensim.models import Phrases
from gensim.models.word2vec import LineSentence

In [445]:
intermediate_directory = '../corpus_samples/lemmatized/' + language + '/'
def discardable(token):

    return token.is_punct or token.is_space or token.is_stop or token.is_punct or token.like_num

def line_review(filename):
    """
    generator function to read in reviews from the file
    and un-escape the original line breaks in the text
    """
    
    with codecs.open(filename, encoding='utf_8') as f:
        for review in f:
            yield review.replace('\\n', '\n')

def lemmatized_sentence_corpus(filename):
    """
    generator function to use spaCy to parse reviews,
    lemmatize the text, and yield sentences
    """
    
    for parsed_review in nlp.pipe(line_review(filename),
                                  batch_size=10000, n_threads=4):
        
        for sent in parsed_review.sents:
            yield u' '.join([token.lemma_ for token in sent
                             if not discardable(token)])


unigram_sentences_filepath = os.path.join(intermediate_directory,
                                          'unigram_sentences_all.txt')

with codecs.open(unigram_sentences_filepath, 'w', encoding='utf_8') as f:
    for sentence in lemmatized_sentence_corpus(filename):
        f.write(sentence + '\n')

/opt/conda/lib/python3.7/runpy.py:193: DeprecationWarning: [W016] The keyword argument `n_threads` on the is now deprecated, as the v2.x models cannot release the global interpreter lock. Future versions may introduce a `n_process` argument for parallel inference via multiprocessing.
  "__main__", mod_spec)


In [446]:
unigram_sentences = LineSentence(unigram_sentences_filepath)

In [447]:
for unigram_sentence in it.islice(unigram_sentences, 0, 240):
    print(u' '.join(unigram_sentence))
    print(u'')

auf Sternwarte Wien ander durch VON Oarr auf Sternwarte München ausgeführt worden war

Das durch Konstanten gramme val

abs

Wien M.-G. i.

m

cm sek-2 definiert Wiener gramme

System ist bisher allgemein al

Grundlage für relativen gramme

Messungen beibehalten worden näherungsweise ermittelten

Reduktionsbeträge Beobachtungsreihen auf dasselbe haben Herrn Prof

helmert dazu gedient eine diesem System angepaßte Normalformel abzuleiten seit zur Darstellung

Verlaufs Schwerkraft an Erdoberfläche verwendet wird

In oben erwähnten Berichte

wie Herr Prof

helmert jedoch schon darauf hin daß Wiener Fundamentalkonstante nach seinen auf andere

Material gestützten Untersuchungen

wahrscheinlich einer Korrektion

cm sek-2 bedürfe

ihm älteren meist beträchtlichen systematischen Fehlern behafteten

absoluten

Messungen zur

Ableitung eines

Normalwertes gramme nicht

geeignet erschienen so leitete er im jahre eine absolute Bestimmung gramme für Potsdam Wege

bei dieser über jahre ausgedehnten 

In [448]:
bigram_model_filepath = os.path.join(intermediate_directory, 'bigram_model')

In [449]:
bigram_model = Phrases(unigram_sentences)

bigram_model.save(bigram_model_filepath)
    
# load the finished model from disk
bigram_model = Phrases.load(bigram_model_filepath)

In [450]:
bigram_sentences_filepath = os.path.join(intermediate_directory,
                                         'bigram_sentences_all.txt')
with codecs.open(bigram_sentences_filepath, 'w', encoding='utf_8') as f:
    for unigram_sentence in unigram_sentences:
        bigram_sentence = u' '.join(bigram_model[unigram_sentence])
        f.write(bigram_sentence + '\n')

In [451]:
bigram_sentences = LineSentence(bigram_sentences_filepath)

In [452]:
for bigram_sentence in it.islice(bigram_sentences, 0, 240):
    print(u' '.join(bigram_sentence))
    print(u'')

auf Sternwarte Wien ander durch VON Oarr auf Sternwarte München ausgeführt worden war

Das durch Konstanten gramme val

abs

Wien M.-G. i.

m

cm sek-2 definiert Wiener gramme

System ist bisher allgemein al

Grundlage für relativen gramme

Messungen beibehalten worden näherungsweise ermittelten

Reduktionsbeträge Beobachtungsreihen auf dasselbe haben Herrn Prof

helmert dazu gedient eine diesem System angepaßte Normalformel abzuleiten seit zur Darstellung

Verlaufs Schwerkraft an Erdoberfläche verwendet wird

In oben erwähnten Berichte

wie Herr Prof

helmert jedoch schon darauf hin daß Wiener Fundamentalkonstante nach seinen auf andere

Material gestützten Untersuchungen

wahrscheinlich einer Korrektion

cm sek-2 bedürfe

ihm älteren meist beträchtlichen systematischen Fehlern behafteten

absoluten

Messungen zur

Ableitung eines

Normalwertes gramme nicht

geeignet erschienen so leitete er im jahre eine absolute Bestimmung gramme für Potsdam Wege

bei dieser über jahre ausgedehnten 

In [453]:
trigram_model_filepath = os.path.join(intermediate_directory,
                                      'trigram_model_all')

trigram_model = Phrases(bigram_sentences)

trigram_model.save(trigram_model_filepath)
    
# load the finished model from disk
trigram_model = Phrases.load(trigram_model_filepath)

trigram_sentences_filepath = os.path.join(intermediate_directory,
                                          'trigram_sentences_all.txt')


with codecs.open(trigram_sentences_filepath, 'w', encoding='utf_8') as f:
        
    for bigram_sentence in bigram_sentences:
            
        trigram_sentence = u' '.join(trigram_model[bigram_sentence])
            
        f.write(trigram_sentence + '\n')

In [454]:
from gensim.corpora import Dictionary, MmCorpus
from gensim.models.ldamulticore import LdaMulticore

import pyLDAvis
import pyLDAvis.gensim
import warnings
import pickle

In [455]:
trigram_dictionary_filepath = os.path.join(intermediate_directory,
                                           'trigram_dict_all.dict')

In [456]:
#extreme filters hyperparams
no_below = 4
no_above = 0.5

trigram_pages = LineSentence(trigram_sentences_filepath)

# learn the dictionary by iterating over all of the pages
trigram_dictionary = Dictionary(trigram_pages)
    
# filter tokens that are very rare or too common from
# the dictionary (filter_extremes) and reassign integer ids (compactify)
trigram_dictionary.filter_extremes(no_below, no_above)
trigram_dictionary.compactify()

trigram_dictionary.save(trigram_dictionary_filepath)
    
# load the finished dictionary from disk
trigram_dictionary = Dictionary.load(trigram_dictionary_filepath)

In [457]:
trigram_bow_filepath = os.path.join(intermediate_directory,
                                    'trigram_bow_corpus_all.mm')

In [458]:
def trigram_bow_generator(filepath):
    """
    generator function to read pages from a file
    and yield a bag-of-words representation
    """
    
    for review in LineSentence(filepath):
        yield trigram_dictionary.doc2bow(review)

In [459]:
MmCorpus.serialize(trigram_bow_filepath,
                       trigram_bow_generator(trigram_sentences_filepath))
    
# load the finished bag-of-words corpus from disk
trigram_bow_corpus = MmCorpus(trigram_bow_filepath)

In [460]:
lda_model_filepath = os.path.join(intermediate_directory, 'lda_model_all')

In [461]:
%%time

with warnings.catch_warnings():
    warnings.simplefilter('ignore')
        
    # workers => sets the parallelism, and should be
        # set to your number of physical cores minus one
    lda = LdaMulticore(trigram_bow_corpus,
                           num_topics=3,
                           id2word=trigram_dictionary,
                           workers=3)
    
lda.save(lda_model_filepath)
    
# load the finished LDA model from disk
lda = LdaMulticore.load(lda_model_filepath)

CPU times: user 80 ms, sys: 340 ms, total: 420 ms
Wall time: 637 ms


In [462]:
def explore_topic(topic_number, topn=25):
    """
    accept a user-supplied topic number and
    print out a formatted list of the top terms
    """
        
    print(u'{:20} {}'.format(u'term', u'frequency') + u'\n')

    for term, frequency in lda.show_topic(topic_number, topn=25):
        print(u'{:20} {:.3f}'.format(term, round(frequency, 3)))

In [463]:
explore_topic(topic_number=2)

term                 frequency

eine                 0.077
dieser               0.076
gramme               0.074
im                   0.046
er                   0.044
jahre                0.044
über                 0.043
Kommission           0.038
Prof                 0.037
docteur              0.035
cm                   0.033
Herrn                0.030
schon                0.027
al                   0.027
VON                  0.026
Wien                 0.025
durch                0.025
Berlin               0.024
statt                0.024
sich                 0.020
Potsdam              0.020
Herr                 0.020
Permanenten          0.020
ein                  0.019
Wiener               0.019


In [464]:
LDAvis_data_filepath = os.path.join(intermediate_directory, 'ldavis_prepared')

In [465]:
LDAvis_prepared = pyLDAvis.gensim.prepare(lda, trigram_bow_corpus,
                                              trigram_dictionary)

/opt/conda/lib/python3.7/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


In [466]:
pyLDAvis.display(LDAvis_prepared)